In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

fasta_files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if not"fasta" in filename:
            print(os.path.join(dirname, filename))
        else:
            fasta_files.append(filename)
print(f"{len(fasta_files)} fasta files")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stanford-rna-3d-folding/sample_submission.csv
/kaggle/input/stanford-rna-3d-folding/validation_sequences.csv
/kaggle/input/stanford-rna-3d-folding/test_sequences.csv
/kaggle/input/stanford-rna-3d-folding/validation_labels.csv
/kaggle/input/stanford-rna-3d-folding/train_labels.csv
/kaggle/input/stanford-rna-3d-folding/train_sequences.csv
856 fasta files


In [2]:
submission = pd.read_csv("/kaggle/input/stanford-rna-3d-folding/sample_submission.csv")
submission.head()

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
0,R1107_1,G,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,R1107_2,G,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,R1107_3,G,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,R1107_4,G,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,R1107_5,G,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
import pandas as pd
import numpy as np

# Read the sample submission
df = pd.read_csv("/kaggle/input/stanford-rna-3d-folding/sample_submission.csv")

# Identify the columns that need random values
# (Here, we assume columns are x_1, y_1, z_1, ..., x_5, y_5, z_5)
coord_cols = []
for i in range(1, 6):  # 5 structures
    for axis in ["x", "y", "z"]:
        col_name = f"{axis}_{i}"
        if col_name in df.columns:
            coord_cols.append(col_name)

# Generate random coordinates for each coordinate column
# You can adjust the range or distribution as desired
for col in coord_cols:
    df[col] = np.random.uniform(-20, 20, len(df))  # random floats in [-20, 20]

# Save your random predictions to submission.csv
df.to_csv("submission.csv", index=False)
print("Random submission file created: submission.csv")


Random submission file created: submission.csv


In [4]:
df

,ID,resname,resid,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5
0,R1107_1,G,1,10.680066,1.310653,14.610036,15.347600,-15.736185,-16.190976,4.700351,12.493449,15.814823,4.492099,14.671359,-17.963680,1.503123,14.685762,14.657184
1,R1107_2,G,2,3.340223,-19.572080,15.193208,17.138698,-6.962406,-10.103336,11.833461,-6.595206,-10.241788,-11.460545,-17.193260,-16.942206,-18.997491,14.461989,-8.423967
2,R1107_3,G,3,-13.930460,0.636253,7.792933,-10.500239,-17.610435,-0.768651,8.675312,5.384008,15.851660,8.874039,14.278141,-12.279098,-10.376189,-12.105123,-12.679030
3,R1107_4,G,4,-3.622042,9.567275,8.032671,-4.921203,9.393641,7.554603,16.009753,-0.967519,-13.384658,16.213174,19.204982,-18.803124,-3.633948,17.549131,-19.230796
4,R1107_5,G,5,-10.046072,-15.660565,-13.879088,7.182384,5.480743,12.092154,-10.278019,-14.484375,-13.323995,-11.796731,10.683036,-19.513564,-14.604134,-13.099115,1.307363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,R1189_114,U,114,19.013365,10.292092,-4.036909,4.418269,0.847563,10.995022,19.805172,6.820840,18.450082,-4.135302,-1.376033,-16.542536,-16.919285,6.707994,-16.764699
2511,R1189_115,U,115,-13.495827,9.724542,1.310777,4.794731,-8.488408,4.443855,9.181135,-0.971080,-1.294591,-8.514624,0.445522,2.253015,-4.614165,-18.631784,-0.290757
2512,R1189_116,U,116,-4.568112,-17.619740,0.766868,11.500103,19.594107,15.658589,16.898344,19.996582,5.443810,4.898586,-5.414997,9.639271,13.879094,-7.528065,-5.152600
2513,R1189_117,U,117,-13.859197,-10.033386,15.150263,-3.006937,-10.852147,2.755628,-3.924326,-12.037366,-14.873615,-10.096562,2.668361,-0.468482,16.823525,-13.667449,17.075220
